In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
from tensorflow.keras import layers, models # type: ignore
from sklearn.model_selection import train_test_split
import pickle

ModuleNotFoundError: No module named 'cv2'

In [2]:
# Function to train the CNN model (or load a pre-trained one)
def build_and_train_model():
    (x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data() #Load Data
    x_train_full = x_train_full.astype('float32') / 255.0 #Normalize //from 0 to 1
    x_test = x_test.astype('float32') / 255.0 #Normalize
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

    x_train = x_train.reshape(-1, 28, 28, 1)
    x_val = x_val.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    # Build the CNN model
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.6), #to prevent overfitting
        layers.Dense(10, activation='softmax')  # 10 classes for digits 0-9
    ])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

    # Save the model
    model.save('attendance_digit_model.h5')

In [2]:
from tensorflow.keras.models import load_model 
#build_and_train_model()
model = load_model('./attendance_digit_model3.h5')

In [6]:
from PIL import Image

def load_and_preprocess_image(path):
    img = Image.open(path)
    #.convert('L')           # Convert to grayscale
    #img = img.resize((28, 28))                    # Resize to 28x28
    # img_array = np.array(img) / 255.0             # Normalize
    # img_array = 1 - img_array                     # Invert colors if white digit on black bg
    # img_array = img_array.reshape(1, 28, 28, 1)   # Add batch and channel dims
    return img

def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in sorted(os.listdir(folder)):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            path = os.path.join(folder, filename)
            img = Image.open(path).convert("L")
            # img = np.array(img) / 255.0  # normalize
            # img = 1 - img  # invert
            # img = img.reshape(28, 28, 1)
            images.append(img)
            filenames.append(filename)
    return np.stack(images), filenames  # <- stack into one tensor


In [10]:
import os
# Load all images
image_folder = "../images/final_clean_cells/"
images, filenames = load_images_from_folder(image_folder)

# # Function to display images
# def display_images(images, titles, rows, cols, figsize=(15, 5)):
#     fig, axes = plt.subplots(rows, cols, figsize=figsize)
#     for i, (img, title) in enumerate(zip(images, titles)):
#         ax = axes[i//cols, i%cols] if rows > 1 else axes[i]
#         ax.imshow(img, cmap='gray')
#         ax.set_title(title)
#         ax.axis('off')
#     plt.tight_layout()
#     plt.show()

# # Display first 5 images
# print("First 5 images:")
# display_images(images[:5], [f"Image {i+1}" for i in range(5)], 1, 5)

# # Display last 5 images
# print("\nLast 5 images:")
# display_images(images[-5:], [f"Image {len(images)-4+i}" for i in range(5)], 1, 5)

# Predict on the full batch
prediction = model.predict(images)
predicted_digits = np.argmax(prediction, axis=1)

# Output results
for fname, digit in zip(filenames, predicted_digits):
    print(f"{fname} ➤ {digit}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
id_cell_001.png ➤ 0
id_cell_002.png ➤ 0
id_cell_003.png ➤ 0
id_cell_004.png ➤ 0
id_cell_005.png ➤ 0
id_cell_006.png ➤ 0
id_cell_007.png ➤ 0
id_cell_008.png ➤ 0
id_cell_009.png ➤ 0
id_cell_010.png ➤ 0
id_cell_011.png ➤ 0
id_cell_012.png ➤ 0
id_cell_013.png ➤ 0
id_cell_014.png ➤ 0
id_cell_015.png ➤ 0
id_cell_016.png ➤ 0
id_cell_017.png ➤ 1
id_cell_018.png ➤ 1
id_cell_019.png ➤ 1
id_cell_020.png ➤ 1
id_cell_021.png ➤ 1
id_cell_022.png ➤ 1
id_cell_023.png ➤ 1
id_cell_024.png ➤ 7
id_cell_025.png ➤ 9
id_cell_026.png ➤ 1
id_cell_027.png ➤ 1
id_cell_028.png ➤ 1
id_cell_029.png ➤ 1
id_cell_030.png ➤ 1
id_cell_031.png ➤ 1
id_cell_032.png ➤ 1
id_cell_033.png ➤ 2
id_cell_034.png ➤ 2
id_cell_035.png ➤ 2
id_cell_036.png ➤ 2
id_cell_037.png ➤ 2
id_cell_038.png ➤ 2
id_cell_039.png ➤ 2
id_cell_040.png ➤ 2
id_cell_041.png ➤ 2
id_cell_042.png ➤ 2
id_cell_043.png ➤ 2
id_cell_044.png ➤ 2
id_cell_045.png ➤ 2
id_cell_046.png ➤ 2
id_cell_047.png ➤ 2
id_cell_048.png ➤ 2
id

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

def build_and_train_model3():
    # 1) LOAD & PREPARE
    (x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train_full = x_train_full.astype('float32') / 255.0
    x_test       = x_test.astype('float32')       / 255.0

    x_train, x_val, y_train, y_val = train_test_split(
        x_train_full, y_train_full, test_size=0.2, random_state=42)
    # add channel dim
    x_train = x_train[..., np.newaxis]
    x_val   = x_val[...,   np.newaxis]
    x_test  = x_test[...,  np.newaxis]

    # 2) SET UP AUGMENTER
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        shear_range=0.1
    )
    datagen.fit(x_train)

    # 3) GENERATE ONE “EPOCH” WORTH OF AUGMENTED SAMPLES
    batch_size = 32
    steps = len(x_train) // batch_size
    X_aug_list, y_aug_list = [], []
    aug_iter = datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=False)
    for _ in range(steps):
        xb, yb = next(aug_iter)
        X_aug_list.append(xb)
        y_aug_list.append(yb)
    X_aug = np.vstack(X_aug_list)
    y_aug = np.hstack(y_aug_list)

    # 4) COMBINE RAW + AUGMENTED & SHUFFLE
    X_comb = np.concatenate([x_train, X_aug], axis=0)
    y_comb = np.concatenate([y_train, y_aug], axis=0)
    X_comb, y_comb = shuffle(X_comb, y_comb, random_state=42)

    # 5) BUILD MODEL
    model = models.Sequential([
        layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(28,28,1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Conv2D(256, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # 6) CALLBACKS
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # 7) TRAIN on combined dataset
    model.fit(
        X_comb, y_comb,
        batch_size=64,
        epochs=10,
        validation_data=(x_val, y_val),
        callbacks=[early_stop, reduce_lr]
    )

    # 8) SAVE & EVAL
    model.save('attendance_digit_model3.h5')
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

In [11]:
build_and_train_model_on_combined_data(r'C:\Users\Fares\Downloads\GitHub\Scanvas\apps\ocr-service\CNN_Model')
# Load the model

Epoch 1/10


C:\Users\Fares\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1908/1908 ━━━━━━━━━━━━━━━━━━━━ 77s 39ms/step - accuracy: 0.8554 - loss: 0.4928
Epoch 2/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 62s 32ms/step - accuracy: 0.9747 - loss: 0.0839
Epoch 3/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 62s 32ms/step - accuracy: 0.9811 - loss: 0.0609
Epoch 4/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 62s 32ms/step - accuracy: 0.9853 - loss: 0.0481
Epoch 5/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 62s 32ms/step - accuracy: 0.9894 - loss: 0.0369
Epoch 6/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 62s 32ms/step - accuracy: 0.9908 - loss: 0.0306
Epoch 7/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 62s 32ms/step - accuracy: 0.9927 - loss: 0.0242
Epoch 8/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 62s 33ms/step - accuracy: 0.9937 - loss: 0.0200
Epoch 9/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 64s 34ms/step - accuracy: 0.9941 - loss: 0.0178
Epoch 10/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 70s 36ms/step - accuracy: 0.9940 - loss: 0.0182
